The following procedure was used for repredicting the mislabeled training data. This was the first step of our two-stage model. 

- The 20 melspectrogram coefficients were previously generated in preprocessing.ipynb from the training data. 

- We trained a random forest classifier purely on the manually verified labels in the training data. 

- We then repredicted the non-manually verified labels in the training data. Using this augmeneted training data set, we would later feed this into a convolutional neural net. 

In [0]:
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd

mel_features = pd.read_csv('/content/drive/Shared drives/301FP/Data_Folder/melfeatures_traindata_five_second.csv')
mel_features = mel_features.rename(columns={'filename': 'fname'})
mel_features

,Unnamed: 0,fname,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20
0,0,4fd883bb.wav,0.241687,0.044887,9125.870779,1672.702388,10248.825906,0.789746,-579.723300,-18.308407,125.062320,-53.604836,89.521860,-37.015236,54.484050,-16.556002,23.986645,-4.907334,-3.249422,-2.603389,-14.892915,2.606663,-10.157707,1.747104,-13.306069,-5.115225,-5.815826,-0.313843
1,0,2137ed0e.wav,0.174657,0.041813,9369.303383,1584.221928,10258.840942,0.844779,-577.944300,-11.396376,134.391530,-48.564130,99.969140,-28.425358,58.554928,-8.056785,30.774693,8.163332,9.344813,8.371096,-5.326722,6.941596,-1.474563,10.421943,0.006369,5.884386,0.589444,8.156341
2,0,d5fc9f70.wav,0.173477,0.054140,8999.263185,1747.530034,10223.336792,0.793233,-457.899050,-7.843943,82.921670,-37.935658,59.493490,-41.937706,37.132866,-28.536560,15.416706,-17.226141,5.781776,5.504612,5.347375,10.525132,-4.152859,6.583116,-11.240765,5.272945,-0.677570,11.292308
3,0,c40632d7.wav,0.188477,0.249545,5599.749900,2689.094943,8897.001139,0.404037,-193.691220,16.029253,10.111848,-18.963854,18.571892,-26.087526,16.465628,-15.243033,16.524448,-5.513516,8.017700,-6.428593,-1.314647,-3.011143,-1.311669,2.210048,-5.851652,3.894205,-0.371223,9.584037
4,0,be014d8c.wav,0.165962,0.041922,9714.555324,660.778533,10264.362346,0.875241,-698.109100,-80.617380,124.515680,-81.507706,86.782906,-47.108845,46.411820,-28.686516,5.874631,-6.310213,-4.807982,7.486679,-7.633713,6.821452,2.367679,14.367212,9.809677,10.471217,-3.430493,-0.628645
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2752,0,0ef9a602.wav,0.196744,0.042528,9331.327445,1187.632188,10241.001057,0.862982,-568.720460,-99.034134,56.000217,-59.200310,60.295210,-52.211730,33.081825,-24.278006,21.866594,5.475962,10.724299,8.348409,-7.900388,4.485754,-3.762764,-0.559823,-11.566172,-3.164255,6.330161,22.486217
2753,0,06896eb5.wav,0.504690,0.272527,2770.721162,3218.626711,6964.794922,0.121279,24.645168,99.578140,8.007272,9.731167,6.521157,-0.088885,11.366303,-15.602901,5.923294,-5.813812,6.624196,-8.281478,0.872738,0.257321,4.888622,4.222697,4.720238,2.973900,-0.039336,3.255640
2754,0,f964eaba.wav,0.312739,0.215331,4760.873640,4406.677992,9991.406250,0.187356,-272.682800,110.255300,106.393560,-26.665146,50.729140,-17.346119,36.185425,-15.532947,32.815544,-8.728172,13.770216,4.448390,7.329053,5.939463,1.075252,11.510622,-5.086331,5.365305,-10.342467,1.231437
2755,0,5ef1bff4.wav,0.581604,0.103921,4502.787906,3754.601724,9557.053630,0.405441,-95.424020,88.546890,12.154624,-15.150885,20.831997,-24.766314,17.725044,-15.148234,14.024963,-7.993412,6.060193,-9.251913,3.240967,-5.214523,-2.687906,0.744879,-8.152694,-0.093921,-10.147766,2.812814


In [0]:
train_labels = pd.read_csv('/content/drive/Shared drives/301FP/Data_Folder/train.csv')
train_labels

,fname,label,manually_verified
0,00044347.wav,Hi-hat,0
1,001ca53d.wav,Saxophone,1
2,002d256b.wav,Trumpet,0
3,00353774.wav,Cello,1
4,003b91e8.wav,Cello,0
...,...,...,...
2752,ff9c6c3f.wav,Trumpet,0
2753,ffc92b01.wav,Cello,1
2754,ffd6b26c.wav,Fart,1
2755,ffe66841.wav,Burping_or_eructation,0


In [0]:
mel_features = pd.merge(train_labels, mel_features, on='fname')

In [0]:
mel_features

,fname,label,manually_verified,Unnamed: 0,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20
0,00044347.wav,Hi-hat,0,0,0.328186,0.059034,7384.509836,3005.085439,10037.413534,0.722898,-223.685640,-52.205780,-31.073942,-25.024858,-6.323498,-44.853428,-5.298163,-37.901363,10.233525,-18.209833,-3.809118,-14.183720,16.731600,30.433992,8.031943,0.023575,-10.989101,-0.887204,-11.423738,6.469756
1,001ca53d.wav,Saxophone,1,0,0.160103,0.110941,5837.442226,4244.907616,10092.243449,0.277235,-273.514500,24.834526,-9.501972,-43.635857,32.866100,-61.312960,23.893885,-44.131380,-5.999252,-30.727200,-14.341383,-13.528973,-10.104232,-14.404510,-14.698747,-5.244896,-4.568790,10.646997,-1.963713,13.948968
2,002d256b.wav,Trumpet,0,0,0.172349,0.041146,9506.065128,1314.184206,10259.437963,0.837839,-540.389900,-3.182828,99.808980,-80.966774,69.165310,-52.619550,38.242313,-20.421453,17.700900,3.152324,16.437110,12.791654,8.106652,11.819880,-6.597773,10.032980,-5.834199,2.872484,-2.533994,8.286274
3,00353774.wav,Cello,1,0,0.167330,0.082485,6927.222909,3588.505076,10143.818567,0.464300,-296.447020,36.684190,33.442250,-33.065792,47.174778,-45.936455,32.766754,-12.169255,18.069307,-6.507904,15.346503,-2.740616,-3.733625,-7.141095,-6.035097,5.733470,-5.982800,6.074183,-11.676677,3.509564
4,003b91e8.wav,Cello,0,0,0.273497,0.074600,6544.083422,4430.596241,10162.824504,0.364393,-308.398300,105.525950,70.852196,-32.598015,79.615550,-50.747600,33.613583,-29.995466,25.607992,-15.407661,4.735444,-3.124808,-23.991749,3.130228,-12.226490,-1.281843,-15.426494,-17.130700,-15.996727,4.919356
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2752,ff9c6c3f.wav,Trumpet,0,0,0.177653,0.170725,4958.169350,3708.872798,9869.384766,0.235107,-214.215790,-57.647970,-60.504547,-57.796540,-13.099021,-44.668736,-12.745372,-31.859549,-2.084733,-20.997484,6.473208,-1.798303,17.456495,22.358340,23.414639,10.459783,-20.593412,-15.420361,-1.212103,18.856290
2753,ffc92b01.wav,Cello,1,0,0.168539,0.074390,7422.976144,3251.686937,10182.758123,0.510007,-394.890780,46.366200,74.653120,-44.474106,80.865300,-32.116840,36.788250,-31.939043,20.649702,-9.251974,1.374114,-9.739198,-16.526970,-3.264557,-10.805027,2.881197,-11.491906,4.598462,-11.217049,-3.457764
2754,ffd6b26c.wav,Fart,1,0,0.446806,0.143721,5796.619604,2919.464370,9039.709273,0.516217,-61.858963,-20.864511,14.133443,27.345268,22.134485,-0.974552,-18.398706,1.215535,16.444494,5.570326,-3.215313,-8.142345,-3.403784,3.399372,7.353151,-1.678882,-7.237845,-8.974787,-18.095354,7.148603
2755,ffe66841.wav,Burping_or_eructation,0,0,0.286055,0.219438,3274.140757,3315.115085,8138.417455,0.195826,-30.428593,84.076416,-6.388475,-5.277185,-12.304390,-29.966839,18.083015,-25.462603,-7.081358,-6.411342,22.581139,3.922271,-3.364579,-25.356718,8.713880,-0.828161,-15.307866,4.441853,-6.727560,0.112131


In [0]:
#### Be Careful around this block: the 'label' column of the df is being mapped from strings to numbers 
labels = mel_features['label'].unique()
d_label = dict(zip(labels, range(0,10)))
reverse_label = dict(zip(range(0,10), labels))
mel_features['label'] = mel_features['label'].map(lambda x: d_label[x], na_action = 'ignore')

In [0]:
mel_features

,fname,label,manually_verified,Unnamed: 0,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20
0,00044347.wav,0,0,0,0.328186,0.059034,7384.509836,3005.085439,10037.413534,0.722898,-223.685640,-52.205780,-31.073942,-25.024858,-6.323498,-44.853428,-5.298163,-37.901363,10.233525,-18.209833,-3.809118,-14.183720,16.731600,30.433992,8.031943,0.023575,-10.989101,-0.887204,-11.423738,6.469756
1,001ca53d.wav,1,1,0,0.160103,0.110941,5837.442226,4244.907616,10092.243449,0.277235,-273.514500,24.834526,-9.501972,-43.635857,32.866100,-61.312960,23.893885,-44.131380,-5.999252,-30.727200,-14.341383,-13.528973,-10.104232,-14.404510,-14.698747,-5.244896,-4.568790,10.646997,-1.963713,13.948968
2,002d256b.wav,2,0,0,0.172349,0.041146,9506.065128,1314.184206,10259.437963,0.837839,-540.389900,-3.182828,99.808980,-80.966774,69.165310,-52.619550,38.242313,-20.421453,17.700900,3.152324,16.437110,12.791654,8.106652,11.819880,-6.597773,10.032980,-5.834199,2.872484,-2.533994,8.286274
3,00353774.wav,3,1,0,0.167330,0.082485,6927.222909,3588.505076,10143.818567,0.464300,-296.447020,36.684190,33.442250,-33.065792,47.174778,-45.936455,32.766754,-12.169255,18.069307,-6.507904,15.346503,-2.740616,-3.733625,-7.141095,-6.035097,5.733470,-5.982800,6.074183,-11.676677,3.509564
4,003b91e8.wav,3,0,0,0.273497,0.074600,6544.083422,4430.596241,10162.824504,0.364393,-308.398300,105.525950,70.852196,-32.598015,79.615550,-50.747600,33.613583,-29.995466,25.607992,-15.407661,4.735444,-3.124808,-23.991749,3.130228,-12.226490,-1.281843,-15.426494,-17.130700,-15.996727,4.919356
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2752,ff9c6c3f.wav,2,0,0,0.177653,0.170725,4958.169350,3708.872798,9869.384766,0.235107,-214.215790,-57.647970,-60.504547,-57.796540,-13.099021,-44.668736,-12.745372,-31.859549,-2.084733,-20.997484,6.473208,-1.798303,17.456495,22.358340,23.414639,10.459783,-20.593412,-15.420361,-1.212103,18.856290
2753,ffc92b01.wav,3,1,0,0.168539,0.074390,7422.976144,3251.686937,10182.758123,0.510007,-394.890780,46.366200,74.653120,-44.474106,80.865300,-32.116840,36.788250,-31.939043,20.649702,-9.251974,1.374114,-9.739198,-16.526970,-3.264557,-10.805027,2.881197,-11.491906,4.598462,-11.217049,-3.457764
2754,ffd6b26c.wav,5,1,0,0.446806,0.143721,5796.619604,2919.464370,9039.709273,0.516217,-61.858963,-20.864511,14.133443,27.345268,22.134485,-0.974552,-18.398706,1.215535,16.444494,5.570326,-3.215313,-8.142345,-3.403784,3.399372,7.353151,-1.678882,-7.237845,-8.974787,-18.095354,7.148603
2755,ffe66841.wav,7,0,0,0.286055,0.219438,3274.140757,3315.115085,8138.417455,0.195826,-30.428593,84.076416,-6.388475,-5.277185,-12.304390,-29.966839,18.083015,-25.462603,-7.081358,-6.411342,22.581139,3.922271,-3.364579,-25.356718,8.713880,-0.828161,-15.307866,4.441853,-6.727560,0.112131


In [0]:
is_manually_verified_idcs = mel_features['manually_verified'] == 1
not_manually_verified_idcs = mel_features['manually_verified'] == 0

manually_verified = mel_features[is_manually_verified_idcs]
not_manually_verified = mel_features[not_manually_verified_idcs]
man_verified_labels = manually_verified['label'].to_numpy()
 
 # make the features matrix for manually and not manually verified 
man_verified_features = manually_verified.iloc[:, 4:].to_numpy()
not_man_verified_features = not_manually_verified.iloc[:,4:].to_numpy()

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics


In [0]:
# X_train, X_test, y_train, y_test = train_test_split(mel_features, labels, test_size = 0.25, random_state = 42)
clf=RandomForestClassifier(n_estimators=1000)

In [0]:
clf.fit(man_verified_features,man_verified_labels)
y_pred=clf.predict(not_man_verified_features)

# print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [0]:
import numpy as np

In [0]:
not_manually_verified['label'] = y_pred
reverified = pd.concat([manually_verified, not_manually_verified], axis=0, sort=True)
reverified['label'] = reverified['label'].apply(lambda x: reverse_label[x])

reverified.to_csv('/content/drive/Shared drives/301FP/Data_Folder/Reverified.csv')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
reverified

,Unnamed: 0,chroma_stft,fname,label,manually_verified,mfcc1,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc2,mfcc20,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,rmse,rolloff,spectral_bandwidth,spectral_centroid,zero_crossing_rate
1,0,0.160103,001ca53d.wav,Saxophone,1,-273.514500,-30.727200,-14.341383,-13.528973,-10.104232,-14.404510,-14.698747,-5.244896,-4.568790,10.646997,-1.963713,24.834526,13.948968,-9.501972,-43.635857,32.866100,-61.312960,23.893885,-44.131380,-5.999252,0.110941,10092.243449,4244.907616,5837.442226,0.277235
3,0,0.167330,00353774.wav,Cello,1,-296.447020,-6.507904,15.346503,-2.740616,-3.733625,-7.141095,-6.035097,5.733470,-5.982800,6.074183,-11.676677,36.684190,3.509564,33.442250,-33.065792,47.174778,-45.936455,32.766754,-12.169255,18.069307,0.082485,10143.818567,3588.505076,6927.222909,0.464300
5,0,0.332682,0048fd00.wav,Gunshot_or_gunfire,1,-246.947720,-9.015137,6.675484,-15.799837,-6.659844,-1.950133,-11.304765,0.135504,-10.852476,2.956355,-8.220402,40.116673,3.109028,46.783420,-13.768485,47.423790,-16.447193,30.331030,-12.979489,23.477379,0.143457,9238.701172,2754.728275,6798.125102,0.579199
6,0,0.224366,006f2f32.wav,Hi-hat,1,-462.501980,-12.109037,5.493013,-4.471132,-11.377826,-2.390957,-10.839288,-5.425712,-6.059890,4.963983,-2.479236,-97.297745,3.718000,37.651054,-43.221200,46.008415,-29.231730,24.668205,-23.732800,4.119383,0.046191,10171.488723,1594.952585,8803.910185,0.823142
7,0,0.214326,0091fc7f.wav,Cello,1,-354.206420,-10.315027,4.963612,-7.533293,-15.916267,-5.839799,-8.465759,2.401174,-11.863917,3.373446,-9.218606,56.348644,-0.743637,58.599632,-50.948277,67.556410,-35.982110,37.816770,-28.814060,17.075563,0.053620,10191.066187,3268.932663,7562.562881,0.629207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2743,0,0.171635,fed181cb.wav,Violin_or_fiddle,0,-631.830500,-4.315804,2.257978,7.300437,-8.105193,6.751427,-0.309794,5.844636,-2.226647,7.073077,2.145866,-78.828470,1.943104,109.377910,-59.933014,71.493220,-47.142834,44.217180,-25.635172,14.359010,0.041583,10265.281677,873.611165,9607.631875,0.862834
2751,0,0.351026,ff875923.wav,Gunshot_or_gunfire,0,-61.186207,-1.627028,10.027098,-13.827254,-2.226257,2.452323,-0.329859,-8.301149,3.641192,16.509356,-6.778057,81.464010,3.598682,-1.387777,18.213648,36.683586,-9.119958,26.117039,-18.124420,20.860783,0.187618,9405.124919,3642.937112,3481.541715,0.179299
2752,0,0.177653,ff9c6c3f.wav,Saxophone,0,-214.215790,-20.997484,6.473208,-1.798303,17.456495,22.358340,23.414639,10.459783,-20.593412,-15.420361,-1.212103,-57.647970,18.856290,-60.504547,-57.796540,-13.099021,-44.668736,-12.745372,-31.859549,-2.084733,0.170725,9869.384766,3708.872798,4958.169350,0.235107
2755,0,0.286055,ffe66841.wav,Burping_or_eructation,0,-30.428593,-6.411342,22.581139,3.922271,-3.364579,-25.356718,8.713880,-0.828161,-15.307866,4.441853,-6.727560,84.076416,0.112131,-6.388475,-5.277185,-12.304390,-29.966839,18.083015,-25.462603,-7.081358,0.219438,8138.417455,3315.115085,3274.140757,0.195826
